# Multilingual Universal Sentence Encoder Q&A Retrieval

**Acknowledgements:**
1. Tutorial on [Colab](https://www.tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa)  
2. Notebook on [Github](https://github.com/tensorflow/hub/blob/master/examples/colab/retrieval_with_tf_hub_universal_encoder_qa.ipynb)  

**References:**  
1. [Universal Encoder Multilinqual Q&A Model](https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3)  
2. SQuAD dataset:
   1. [Home](https://rajpurkar.github.io/SQuAD-explorer/).  
   2. [Retrieval evaluation](https://github.com/google/retrieval-qa-eval)  
3. Simple Neighbors library:  
   1. [Docs](https://simpleneighbors.readthedocs.io/en/latest/)  
   2. [pypi](https://pypi.org/project/simpleneighbors/)  
4. TensorFlow components:
   1. [TF Text (Github guide w/ examples](https://github.com/tensorflow/text)  
   2. [TF Embeddings (tutorial)](https://www.tensorflow.org/tutorials/text/word_embeddings)  
5. NLTK:
   1. [Home](https://www.nltk.org/)  
   2. [Book](https://www.nltk.org/book/)  

**Table of Contents:**  
1. [Example using the model](#Example)  
2. [Tutorial setup](#Setup)  


## Example use

Example of using the [universal-sentence-encoder-multilingual-qa](https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3) model.

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

questions = ['What is your age?']
responses = ['I am 20 years old.', 'good morning']
response_contexts = ['I will be 21 next year.', 'gread day.']

module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')

quesion_embeddings = module.signatures['question_endcoder'](
    tf.constant(quesions))
response_embeddings = module.signatures['response_encoder'](
    input=tf.constant(responses),
    context=tf.constant(response_contexts))

np.inner(question_embeddings['outputs'], response_embeddings['outputs'])

/home/ivogeorg/anaconda3/envs/tf-2/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/ivogeorg/anaconda3/envs/tf-2/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/ivogeorg/anaconda3/envs/tf-2/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


NotFoundError: /home/ivogeorg/anaconda3/envs/tf-2/lib/python3.7/site-packages/tensorflow_text/python/metrics/_text_similarity_metric_ops.so: undefined symbol: _ZN10tensorflow8OpKernel11TraceStringEPNS_15OpKernelContextEb

##### NotFoundError
Probably [due to Conda packaging](https://github.com/tensorflow/text/issues/192)  

```
/home/ivogeorg/anaconda3/envs/tf-2/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
```
followed by  
```
NotFoundError                             Traceback (most recent call last)
<ipython-input-1-63112d34e578> in <module>
      2 import tensorflow_hub as hub
      3 import numpy as np
----> 4 import tensorflow_text
      5 
      6 questions = ['What is your age?']

~/anaconda3/envs/tf-2/lib/python3.7/site-packages/tensorflow_text/__init__.py in <module>
     19 # pylint: disable=wildcard-import
     20 from tensorflow_text.python import keras
---> 21 from tensorflow_text.python import metrics
     22 from tensorflow_text.python.ops import *
     23 

~/anaconda3/envs/tf-2/lib/python3.7/site-packages/tensorflow_text/python/metrics/__init__.py in <module>
     18 
     19 # pylint: disable=wildcard-import
---> 20 from tensorflow_text.python.metrics.text_similarity_metric_ops import *
     21 
     22 # Public symbols in the "tensorflow_text.metrics" package.

~/anaconda3/envs/tf-2/lib/python3.7/site-packages/tensorflow_text/python/metrics/text_similarity_metric_ops.py in <module>
     26 from tensorflow.python.framework import load_library
     27 from tensorflow.python.platform import resource_loader
---> 28 gen_text_similarity_metric_ops = load_library.load_op_library(resource_loader.get_path_to_datafile('_text_similarity_metric_ops.so'))
     29 
     30 

~/anaconda3/envs/tf-2/lib/python3.7/site-packages/tensorflow/python/framework/load_library.py in load_op_library(library_filename)
     56     RuntimeError: when unable to load the library or get the python wrappers.
     57   """
---> 58   lib_handle = py_tf.TF_LoadLibrary(library_filename)
     59   try:
     60     wrappers = _pywrap_python_op_gen.GetPythonWrappers(

NotFoundError: /home/ivogeorg/anaconda3/envs/tf-2/lib/python3.7/site-packages/tensorflow_text/python/metrics/_text_similarity_metric_ops.so: undefined symbol: _ZN10tensorflow8OpKernel11TraceStringEPNS_15OpKernelContextEb
```